In [135]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [136]:
%load_ext autoreload
%autoreload 2 

import os
import sys

curr_dir = os.getcwd()
print('current direcotry is ', curr_dir)
if curr_dir not in sys.path:
    sys.path.append(curr_dir)
src_dir = os.path.join(curr_dir, 'src')
if src_dir not in sys.path:
    sys.path.append(src_dir)
print(sys.path)

#
pdb_dir = './pdb'
data_dir = './data'
outputs_dir = './outputs'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
current direcotry is  /home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex', '/home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex/src']


## view structure

In [137]:
pdb_acc = '1AHW'

In [4]:
# get *pdb
pdb_file = f'pdb/{pdb_acc}.pdb'
if not os.path.isfile(pdb_file):
    print(f"download {pdb_acc} to {pdb_file}")
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_acc}.pdb -P ./pdb")
print(f"{pdb_file} is ready")

download 1AHW to pdb/1AHW.pdb
pdb/1AHW.pdb is ready


In [5]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(f'pdb/{pdb_acc}.pdb').read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex
1AHW contains two copies of antigen-antibody complexes.
chains BA and DE are identified as two antibody pairs, where chains C and F bind to BA and DE, respectively.
Therefoe, 1AHW is split into two files, BAC and DEF (Figure 2A).

In [138]:
# get chains
from src.parse_abag import ParseAbAg

p = ParseAbAg(f'./pdb/{pdb_acc}.pdb', outputs_dir)

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1AHW
outputs dir:  ./outputs/1A/1AHW
[{'chains': [{'AA residues': 214,
              'chain_id': 'A',
              'seq': 'DIKMTQSPSSMYASLGERVTITCKASQDIRKYLNWYQQKPWKSPKTLIYYATSLADGVPSRFSGSGSGQDYSLTISSLESDDTATYYCLQHGESPYTFGGGTKLEINRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC'},
             {'AA residues': 214,
              'chain_id': 'B',
              'seq': 'EIQLQQSGAELVRPGALVKLSCKASGFNIKDYYMHWVKQRPEQGLEWIGLIDPENGNTIYDPKFQGKASITADTSSNTAYLQLSSLTSEDTAVYYCARDNSYYFDYWGQGTTLTVSSAKTTPPSVYPLAPGSAAQTNSMVTLGCLVKGYFPEPVTVTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVPSSTWPSETVTCNVAHPASSTKVDKKI'},
             {'AA residues': 200,
              'chain_id': 'C',
              'seq': 'TNTVAAYNLTWKSTNFKTILEWEPKPVNQVYTVQISTKSGDWKSKCFYTTDTECDLTDEIVKDVKQTYLARVFSYPAGNEPLYENSPEFTPYLETNLGQPTIQSFEQVGTKVNVTVEDERTLVRRNNTFLSLRDVFGKDLIYTLYYWKSSSSGKKTAKTNTNEFLIDVDKGENYCFSVQAVIPSRTVNRKSTDSPVECMG'},
             {'AA residues'

In [68]:
p.structure.header['keywords']

'blood coagulation, tissue factor, fab, complex, antibody, complex (immunoglobulin-tissue factor), complex (immunoglobulin-tissue factor) complex'

In [69]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'immunoglobulin fab 5g9 (light chain)',
  'chain': 'a, d',
  'fragment': 'light chain residues 1 - 214',
  'synonym': 'fab, fab light chain, fab heavy chain'},
 '2': {'misc': '',
  'molecule': 'immunoglobulin fab 5g9 (heavy chain)',
  'chain': 'b, e',
  'fragment': 'heavy chain residues 1 - 214',
  'synonym': 'fab, fab light chain, fab heavy chain'},
 '3': {'misc': '',
  'molecule': 'tissue factor',
  'chain': 'c, f',
  'fragment': 'extracellular domain',
  'synonym': 'tf, thromboplastin, coagulation factor iii',
  'engineered': 'yes'}}

In [139]:
# filter: is antibody-antigen complex
info = p.filter_antibody_antigen()
print(info)
p.update_chains('chain_type')

[{'B': 'heavy', 'C': 'receptor'}, {'B': 'heavy', 'F': 'receptor'}, {'E': 'heavy', 'C': 'receptor'}, {'E': 'heavy', 'F': 'receptor'}, {'A': 'light', 'C': 'receptor'}, {'A': 'light', 'F': 'receptor'}, {'D': 'light', 'C': 'receptor'}, {'D': 'light', 'F': 'receptor'}]


In [140]:
# split chains
p.chains_to_pdb()

Saved chain ['B'] to ./outputs/1A/1AHW/1AHW_B.pdb
Saved chain ['C'] to ./outputs/1A/1AHW/1AHW_C.pdb
Saved chain ['B', 'C'] to ./outputs/1A/1AHW/1AHW_B_C.pdb
Saved chain ['F'] to ./outputs/1A/1AHW/1AHW_F.pdb
Saved chain ['B', 'F'] to ./outputs/1A/1AHW/1AHW_B_F.pdb
Saved chain ['E'] to ./outputs/1A/1AHW/1AHW_E.pdb
Saved chain ['E', 'C'] to ./outputs/1A/1AHW/1AHW_E_C.pdb
Saved chain ['E', 'F'] to ./outputs/1A/1AHW/1AHW_E_F.pdb
Saved chain ['A'] to ./outputs/1A/1AHW/1AHW_A.pdb
Saved chain ['A', 'C'] to ./outputs/1A/1AHW/1AHW_A_C.pdb
Saved chain ['A', 'F'] to ./outputs/1A/1AHW/1AHW_A_F.pdb
Saved chain ['D'] to ./outputs/1A/1AHW/1AHW_D.pdb
Saved chain ['D', 'C'] to ./outputs/1A/1AHW/1AHW_D_C.pdb
Saved chain ['D', 'F'] to ./outputs/1A/1AHW/1AHW_D_F.pdb


In [123]:
p.bounded

[{'type': 'heavy-receptor',
  'chains': [['B'], ['C'], ['B', 'C']],
  'pdb_files': ['./outputs/1A/1AHW/1AHW_B.pdb',
   './outputs/1A/1AHW/1AHW_C.pdb',
   './outputs/1A/1AHW/1AHW_B_C.pdb']},
 {'type': 'heavy-receptor',
  'chains': [['B'], ['F'], ['B', 'F']],
  'pdb_files': ['./outputs/1A/1AHW/1AHW_B.pdb',
   './outputs/1A/1AHW/1AHW_F.pdb',
   './outputs/1A/1AHW/1AHW_B_F.pdb']},
 {'type': 'heavy-receptor',
  'chains': [['E'], ['C'], ['E', 'C']],
  'pdb_files': ['./outputs/1A/1AHW/1AHW_E.pdb',
   './outputs/1A/1AHW/1AHW_C.pdb',
   './outputs/1A/1AHW/1AHW_E_C.pdb']},
 {'type': 'heavy-receptor',
  'chains': [['E'], ['F'], ['E', 'F']],
  'pdb_files': ['./outputs/1A/1AHW/1AHW_E.pdb',
   './outputs/1A/1AHW/1AHW_F.pdb',
   './outputs/1A/1AHW/1AHW_E_F.pdb']},
 {'type': 'light-receptor',
  'chains': [['A'], ['C'], ['A', 'C']],
  'pdb_files': ['./outputs/1A/1AHW/1AHW_A.pdb',
   './outputs/1A/1AHW/1AHW_C.pdb',
   './outputs/1A/1AHW/1AHW_A_C.pdb']},
 {'type': 'light-receptor',
  'chains': [['A'], ['

## load SASA

heavy B + light A ~ receptor C
heavy D + light F ~ receptor F

In [141]:
# calculate SASA
p.build_freesasa()

'./outputs/1A/1AHW/freesasa.p'

In [142]:
from analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa('./outputs/1A/1AHW/freesasa.p')
p2.print_meta()

number of complex 5
heavy chain B ~ receptor chain C
heavy chain E ~ receptor chain F
light chain A ~ receptor chain C
light chain A ~ receptor chain F
light chain D ~ receptor chain F


In [143]:
# complex
p2.print_delta_sasa(0)

complex: heavy chain B ~ receptor chain C
------------------------------
heavy chain:
    chain_id res_no  res aa       value
26         B     27  PHE  F    2.703881
29         B     30  LYS  K    4.996640
30         B     31  ASP  D   84.514522
31         B     32  TYR  Y   51.088643
32         B     33  TYR  Y   69.747366
49         B     50  LEU  L   12.167464
51         B     52  ASP  D   26.934709
53         B     54  GLU  E   35.428922
54         B     55  ASN  N   48.367081
56         B     57  ASN  N   56.514110
57         B     58  THR  T   10.992609
58         B     59  ILE  I   60.837319
59         B     60  TYR  Y    1.161334
64         B     65  GLN  Q   17.123288
98         B     99  ASP  D   12.305995
99         B    100  ASN  N   33.431748
100        B    101  SER  S   78.368512
101        B    102  TYR  Y  105.799768
------------------------------
receptor chain:
    chain_id res_no  res aa       value
137        C    149  LYS  K   19.515373
138        C    150  ASP  D

In [144]:
# complex
p2.print_delta_sasa(1)

complex: heavy chain E ~ receptor chain F
------------------------------
heavy chain:
    chain_id res_no  res aa       value
0          E      1  GLU  E    0.999328
1          E      2  ILE  I    8.111642
25         E     26  GLY  G    1.998656
26         E     27  PHE  F    6.759702
27         E     28  ASN  N    2.513274
29         E     30  LYS  K    3.997312
30         E     31  ASP  D   87.656668
31         E     32  TYR  Y   52.666124
32         E     33  TYR  Y   78.874409
49         E     50  LEU  L    9.463583
51         E     52  ASP  D   29.524386
53         E     54  GLU  E   31.909835
54         E     55  ASN  N   46.289043
56         E     57  ASN  N   59.217991
57         E     58  THR  T   11.154614
58         E     59  ILE  I   67.597021
59         E     60  TYR  Y    2.322667
64         E     65  GLN  Q   13.477282
97         E     98  ARG  R    4.645335
98         E     99  ASP  D   13.333874
99         E    100  ASN  N   31.154697
100        E    101  SER  S   84.1

In [145]:
# complex
p2.print_delta_sasa(2)

complex: light chain A ~ receptor chain C
------------------------------
light chain:
   chain_id res_no  res aa      value
0         A      1  ASP  D   6.815396
26        A     27  GLN  Q  18.581339
29        A     30  ARG  R  22.065340
31        A     32  TYR  Y  65.979101
49        A     50  TYR  Y  53.747636
90        A     91  HIS  H  22.959061
91        A     92  GLY  G  23.337158
92        A     93  GLU  E  32.796367
93        A     94  SER  S  67.048926
94        A     95  PRO  P   2.703881
95        A     96  TYR  Y  36.082624
------------------------------
receptor chain:
    chain_id res_no  res aa       value
110        C    122  LYS  K    5.026548
137        C    149  LYS  K   30.904023
140        C    152  ILE  I    8.111642
145        C    157  TYR  Y    7.195152
154        C    166  LYS  K   44.268207
155        C    167  THR  T   52.992636
156        C    168  ALA  A   21.631047
157        C    169  LYS  K  128.940965
158        C    170  THR  T   11.195279
159        

In [146]:
# complex
p2.print_delta_sasa(3)

complex: light chain A ~ receptor chain F
------------------------------
light chain:
    chain_id res_no  res aa       value
11         A     12  TYR  Y    7.528965
104        A    105  GLU  E   10.278789
139        A    140  TYR  Y    1.027879
140        A    141  PRO  P   37.854332
141        A    142  LYS  K    6.187882
142        A    143  ASP  D   80.736971
143        A    144  ILE  I    4.055821
144        A    145  ASN  N   18.413226
146        A    147  LYS  K   11.976857
196        A    197  THR  T   49.270238
197        A    198  HIS  H    9.868283
198        A    199  LYS  K  145.953720
200        A    201  SER  S    2.997984
201        A    202  THR  T  111.953607
202        A    203  SER  S   11.519341
203        A    204  PRO  P   10.815523
------------------------------
receptor chain:
    chain_id res_no  res aa      value
16         F     20  LYS  K  42.951654
54         F     58  ASP  D  17.609582
56         F     60  THR  T  11.843402
57         F     61  ASP  D  17

In [147]:
# complex
p2.print_delta_sasa(4)

complex: light chain D ~ receptor chain F
------------------------------
light chain:
   chain_id res_no  res aa      value
0         D      1  ASP  D  13.148269
26        D     27  GLN  Q  23.226674
29        D     30  ARG  R  20.314493
31        D     32  TYR  Y  63.580092
49        D     50  TYR  Y  54.895599
90        D     91  HIS  H  19.194578
91        D     92  GLY  G  22.829640
92        D     93  GLU  E  40.261294
93        D     94  SER  S  70.885539
94        D     95  PRO  P   6.759702
95        D     96  TYR  Y  34.827797
------------------------------
receptor chain:
    chain_id res_no  res aa       value
110        F    122  LYS  K    5.997275
137        F    149  LYS  K   32.446570
140        F    152  ILE  I    8.111642
145        F    157  TYR  Y    8.223031
153        F    165  LYS  K    1.998656
154        F    166  LYS  K   56.643971
155        F    167  THR  T   49.535728
156        F    168  ALA  A   20.279106
157        F    169  LYS  K  100.509351
158        

In [154]:
# retrieve sequences
motifs = p2.retrieve_seq()
motifs

Sequence-SASA data: ./outputs/1A/1AHW/seq_sasa.p.


([{'heavy': [{'seq': 'FNIKDYYMHWVKQRPEQGLEWIGLIDPENGNTIYDPKFQ',
     'sig_res': 14,
     'start': (26, '27'),
     'end': (64, '65')},
    {'seq': 'DNSY', 'sig_res': 4, 'start': (98, '99'), 'end': (101, '102')}],
   'receptor': [{'seq': 'KDLIYTLYYWKSSSSGKKTAKTNTNEFLIDVDKGENYCFSVQAVIPSRTVNRKSTD',
     'sig_res': 23,
     'start': (137, '149'),
     'end': (192, '204')}]},
  {'heavy': [{'seq': 'EIQLQQSGAELVRPGALVKLSCKASGFNIKDYYMHWVKQRPEQGLEWIGLIDPENGNTIYDPKFQ',
     'sig_res': 18,
     'start': (0, '1'),
     'end': (64, '65')},
    {'seq': 'RDNSY', 'sig_res': 5, 'start': (97, '98'), 'end': (101, '102')}],
   'receptor': [{'seq': 'KDLIYTLYYWKSSSSGKKTAKTNTNEFLIDVDKGENYCFSVQAVIPSRTVNRKSTD',
     'sig_res': 22,
     'start': (137, '149'),
     'end': (192, '204')}]},
  {'light': [{'seq': 'DIKMTQSPSSMYASLGERVTITCKASQDIRKYLNWYQQKPWKSPKTLIYY',
     'sig_res': 5,
     'start': (0, '1'),
     'end': (49, '50')},
    {'seq': 'HGESPY', 'sig_res': 6, 'start': (90, '91'), 'end': (95, '96')}],
   're

## distance

In [166]:
# distance
p.build_dist()

'./outputs/1A/1AHW/dist.p'

In [167]:
from analyze_dist import AnalyzeDist

max_span = 30
p3 = AnalyzeDist('./outputs/1A/1AHW/dist.p', max_span=max_span)
p3.print_meta()

number of complex 8
heavy chain B ~ receptor chain C
heavy chain B ~ receptor chain F
heavy chain E ~ receptor chain C
heavy chain E ~ receptor chain F
light chain A ~ receptor chain C
light chain A ~ receptor chain F
light chain D ~ receptor chain C
light chain D ~ receptor chain F


In [168]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs[0]

Sequence-Distance data: ./outputs/1A/1AHW/seq_dist.p.


[{'B': [{'seq': 'KDYYMHWVKQRPEQGLEWIGLIDPENGN',
    'sig_res': 6,
    'start': (29, 30),
    'end': (56, 57),
    'cutoff': 3},
   {'seq': 'Y',
    'sig_res': 1,
    'start': (101, 102),
    'end': (101, 102),
    'cutoff': 3}],
  'C': [{'seq': 'KDLIYTLYYWKSSSSGKKTAKTNTNEFLIDVDKGENYCFSVQAVIPSRTVNRKSTD',
    'sig_res': 8,
    'start': (137, 149),
    'end': (192, 204),
    'cutoff': 3}]},
 {'B': [{'seq': 'KDYYMHWVKQRPEQGLEWIGLIDPENGNTI',
    'sig_res': 9,
    'start': (29, 30),
    'end': (58, 59),
    'cutoff': 4},
   {'seq': 'NSY',
    'sig_res': 3,
    'start': (99, 100),
    'end': (101, 102),
    'cutoff': 4}],
  'C': [{'seq': 'KDLIYTLYYWKSSSSGKKTAKTNTNEFLIDVDKGENYCFSVQAVIPSRTVNRKSTD',
    'sig_res': 16,
    'start': (137, 149),
    'end': (192, 204),
    'cutoff': 4}]},
 {'B': [{'seq': 'IKDYYMHWVKQRPEQGLEWIGLIDPENGNTIYDPKFQ',
    'sig_res': 12,
    'start': (28, 29),
    'end': (64, 65),
    'cutoff': 5},
   {'seq': 'DNSY',
    'sig_res': 4,
    'start': (98, 99),
    'end': (101,

In [169]:
# complex
p3.print_min_dist(0)

B-C
------------------------------
     res_no  res atom      value aa
0         1  GLU    C   7.314615  E
1         2  ILE    C   6.649883  I
2         3  GLN    C  11.738989  Q
3         4  LEU    C  12.281773  L
4         5  GLN    C  15.947675  Q
..      ...  ...  ...        ... ..
209     210  VAL    C  38.351564  V
210     211  ASP    C  37.646070  D
211     212  LYS    C  41.805687  K
212     213  LYS    C  44.227727  K
213     214  ILE    C  44.867092  I

[214 rows x 5 columns]
------------------------------
     res_no  res atom     value aa
137     149  LYS    C  2.441961  K
138     150  ASP    C  4.614493  D
139     151  LEU    C  4.970476  L
140     152  ILE    C  3.620794  I
142     154  THR    C  2.735831  T
144     156  TYR    C  2.903691  Y
152     164  GLY    C  4.855470  G
153     165  LYS    C  2.906523  K
154     166  LYS    C  4.388762  K
155     167  THR    C  3.067175  T
157     169  LYS    C  3.120922  K
159     171  ASN    C  3.104170  N
178     190  GLN    C  